In [1]:
from preprocessing import Audio
import tensorflow as tf
import tf_train as tft
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sp
import librosa as li
import librosa.display
import IPython.display as ipd

%matplotlib inline
#tf.debugging.set_log_device_placement(True)

In [2]:
GPU = tf.config.list_logical_devices('GPU')
CPU = tf.config.list_logical_devices('CPU')
DEVICE = GPU[0].name if GPU else CPU[0].name
DEVICE

'/device:GPU:0'

<h1>Подготовка данных для обучения и тестирования</h1>

In [3]:
def get_data():
    def norm(x):
        std = np.std(x ,ddof=1)
        return (x - np.mean(x)) / std

    PATH_0 = r'AudioData/0 Данные' # negative class
    PATH_1 = r'AudioData/1 Мира' # positive class

    PATHS = li.util.find_files(PATH_0)[:1000] + li.util.find_files(PATH_1)
    N = len(PATHS)

    np.random.shuffle(PATHS)

    SOUNDS = []
    for path in PATHS:
        SOUNDS.append(Audio(path))
        print('Loading {:.1f}%'.format(len(SOUNDS) / N * 100))
        ipd.clear_output(wait=True)
    DATA = []
    LABELS = []

    for sound in SOUNDS:
        DATA.append(norm(sound.e_parts))
        LABELS.append(sound.label)

    return (np.array(DATA), np.array(LABELS))

In [4]:
data, labels = get_data()

Loading 100.0%


In [88]:
test_data, test_labels = data[8000:], labels[8000:]
data, labels = data[:8000], labels[:8000]

In [5]:
data.shape, labels.shape

((1070, 20), (1070,))

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(1070)
dataset

<BatchDataset shapes: ((None, 20), (None,)), types: (tf.float64, tf.int32)>

In [188]:
data = np.reshape(data, (10070, 20))

In [211]:
labels = np.reshape(labels, (10070,))

In [209]:
labels

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

<h1>Создание модели нейронной сети</h1>

In [52]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01),
                          input_shape=(20,)),
    tf.keras.layers.Dense(2, activation='softmax')
], name='Lada')

model.summary()

Model: "Lada"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 12)                252       
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [11]:
y_pred = model.predict(data)
print(y_pred[:10])
y_pred = np.argmax(y_pred, axis=-1)
print(y_pred[:10])
print(labels[:10])

[[0.24388829 0.7561117 ]
 [0.44321573 0.5567843 ]
 [0.788806   0.21119395]
 [0.1729286  0.82707137]
 [0.42110857 0.57889146]
 [0.56216836 0.43783167]
 [0.14818363 0.8518164 ]
 [0.40375194 0.59624803]
 [0.33486465 0.6651354 ]
 [0.6221114  0.37788862]]
[1 1 0 1 1 0 1 1 1 0]
[0 0 0 1 0 0 0 0 0 0]


# Клонирование модели для transfer learning

In [9]:
model = tf.keras.models.load_model('Джарвис.h5')
model = tf.keras.models.clone_model(model)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                252       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


<h1>Обучение</h1>

In [53]:
tft.train(
    train_dataset=dataset,
    model=model,
    epochs=10000,
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

SparseCategoricalCrossentropy): 0.001, Recall: 0.8433099985122681, Precision: 0.9267842769622803
Epoch 8290 -- Loss(SparseCategoricalCrossentropy): 0.001, Recall: 0.8433321118354797, Precision: 0.9267956614494324
Epoch 8300 -- Loss(SparseCategoricalCrossentropy): 0.001, Recall: 0.8433542251586914, Precision: 0.9268070459365845
Epoch 8310 -- Loss(SparseCategoricalCrossentropy): 0.001, Recall: 0.8433763980865479, Precision: 0.9268183708190918
Epoch 8320 -- Loss(SparseCategoricalCrossentropy): 0.001, Recall: 0.8433985114097595, Precision: 0.9268297553062439
Epoch 8330 -- Loss(SparseCategoricalCrossentropy): 0.001, Recall: 0.8434206247329712, Precision: 0.9268410801887512
Epoch 8340 -- Loss(SparseCategoricalCrossentropy): 0.001, Recall: 0.8434427380561829, Precision: 0.9268524646759033
Epoch 8350 -- Loss(SparseCategoricalCrossentropy): 0.001, Recall: 0.8434648513793945, Precision: 0.9268637895584106
Epoch 8360 -- Loss(SparseCategoricalCrossentropy): 0.001, Recall: 0.8434869050979614, Preci

In [56]:
model.save('models\{}-v1.0.h5'.format(model.name))

In [8]:
model = tf.keras.models.load_model(r'models\Mira-v1.0.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                252       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [9]:
predictions = tft.predict_classes(model=model, features=data)

In [10]:
n = 0

for i in range(len(labels)):
    n += int(labels[i] == predictions[i])

print(n, n / len(labels) * 100)

1070 100.0


In [24]:
predictions[:10], labels[:10]

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int64)>,
 array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0]))

In [39]:
model.predict(data[:2])

array([[9.9999857e-01, 1.4108425e-06],
       [3.2748736e-05, 9.9996722e-01]], dtype=float32)

In [40]:
labels[:2]

array([0, 1])

In [69]:
filename = 'test.tfrecord'
writer_data = tf.data.experimental.TFRecordWriter("datasets\Mira\data.tfrecord")
writer_labels = tf.data.experimental.TFRecordWriter("datasets\Mira\labels.tfrecord")
test_data = tf.data.Dataset.from_tensor_slices(data)
test_labels = tf.data.Dataset.from_tensor_slices(labels)
test_data = test_data.map(lambda x: str(x))
test_labels = test_labels.map(lambda x: str(x))
writer_data.write(test_data)
writer_labels.write(test_labels)

In [75]:
loaded_dataset = tf.data.TFRecordDataset(["datasets\Mira\labels.tfrecord"])

In [76]:
for d in loaded_dataset.take(1):
    print(d)

tf.Tensor(b'Tensor("args_0:0", shape=(), dtype=int32)', shape=(), dtype=string)


In [77]:
loaded_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [83]:
for d in test_data.take(1):
    print(d.numpy())

b'Tensor("args_0:0", shape=(20,), dtype=float64)'


In [84]:
data[0]

array([-1.67636868, -1.63167534, -1.07627448, -0.92766935, -0.09969852,
       -0.79751682, -0.47027632,  1.5961253 ,  1.63171389,  1.30207031,
        0.55290537,  1.14207755, -0.64697944, -0.13645845, -0.15388176,
        0.74642466,  0.9525158 ,  0.10332707, -0.43498627,  0.02462547])